# Import Libraries

In [1]:
# importing libraries

import pandas as pd
import numpy as np
import os

# Import Data

In [2]:
# importing data

path = '/Users/tannu/Desktop/Data Analytics/Instacart Basket Analysis Dec-2025 '
df_cust = pd.read_csv(os.path.join(path, '2 Data', 'Original Data', 'customers.csv'), index_col = False)

In [3]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## Data Wrangling

### Renaming of columns 

In [3]:
# Renaming columns to maintain consistency among all column names
df_cust.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'surname', 'Gender' : 'gender', 'STATE' : 'state', 
                          'Age' : 'age', 'n_dependants' : 'number_of_dependants','fam_status' : 'marital_status'}, inplace = True)

In [5]:
df_cust.head()

,user_id,first_name,surname,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


### Changing Data Types

In [6]:
df_cust.dtypes

user_id                  int64
first_name              object
surname                 object
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependants     int64
marital_status          object
income                   int64
dtype: object

In [4]:
# changing data type of 'user_id' since we do not need this variable for any numerical calculation
df_cust['user_id'] = df_cust['user_id'].astype('category')

In [5]:
df_cust['user_id'].dtype

CategoricalDtype(categories=[     1,      2,      3,      4,      5,      6,      7,
                       8,      9,     10,
                  ...
                  206200, 206201, 206202, 206203, 206204, 206205, 206206,
                  206207, 206208, 206209],
, ordered=False, categories_dtype=int64)

In [9]:
df_cust.describe()

,age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


## Data Quality & Consistency Checks

### Mixed-Type Column

In [10]:
# checking for mixed-type column
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].map(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

first_name


In [6]:
# changing data type of 'first_name' to string 
df_cust['first_name'] = df_cust['first_name'].astype('str')

In [12]:
df_cust['first_name'].dtype 

dtype('O')

In [13]:
# checking if 'first_name' column is still mixed-type
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].map(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

### Missing Values

In [14]:
# checking for missing values
df_cust.isnull().sum()

user_id                 0
first_name              0
surname                 0
gender                  0
state                   0
age                     0
date_joined             0
number_of_dependants    0
marital_status          0
income                  0
dtype: int64

### Duplicates

In [15]:
# checking for duplicates
df_dups = df_cust[df_cust.duplicated()]

In [16]:
df_dups

,user_id,first_name,surname,gender,state,age,date_joined,number_of_dependants,marital_status,income


#### There are no missing values and duplicates in customer dataframe.  

In [17]:
df_cust.shape

(206209, 10)

## Joining customers dataset with ords_prods_latest dataset

In [7]:
# importing data
df_ords_prods = pd.read_pickle(os.path.join(path, '2 Data', 'Prepared Data', 'ords_prods_merge_group.pkl'))

In [19]:
df_ords_prods.head(2)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,...,_merge,price_range,busiest_day,busiest_hours,max_order,loyalty_flag,average_spent,spending_flag,order_frequency,order_frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range product,Regularly Busy,Most Orders,32,Regular Customer,6.935811,Low Spender,8.0,Frequent Customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range product,Regularly Busy,Average Orders,32,Regular Customer,6.935811,Low Spender,8.0,Frequent Customer


#### The two datasets will be joined by the common variable 'user_id'.

In [27]:
# checking data types for all columns
df_ords_prods.dtypes

product_id                  int64
product_name               object
aisle_id                    int64
department_id               int64
prices                    float64
order_id                    int64
user_id                    object
order_number                int64
orders_day_of_the_week      int64
order_hour_of_day           int64
days_since_prior_order    float64
add_to_cart_order           int64
reordered                   int64
price_range                object
busiest_day                object
busiest_hours              object
max_order                   int64
loyalty_flag               object
average_spent             float64
spending_flag              object
order_frequency           float64
order_frequency_flag       object
dtype: object

In [8]:
# changing data type of all categorical data into category to save memory
cols = ['product_id', 'aisle_id', 'department_id', 'order_id', 'user_id', 'orders_day_of_the_week']
df_ords_prods[cols] = df_ords_prods[cols].astype('category')

In [9]:
df_ords_prods.dtypes

product_id                category
product_name                object
aisle_id                  category
department_id             category
prices                     float64
order_id                  category
user_id                   category
order_number                 int64
orders_day_of_the_week    category
order_hour_of_day            int64
days_since_prior_order     float64
add_to_cart_order            int64
reordered                    int64
_merge                    category
price_range                 object
busiest_day                 object
busiest_hours               object
max_order                    int64
loyalty_flag                object
average_spent              float64
spending_flag               object
order_frequency            float64
order_frequency_flag        object
dtype: object

In [23]:
df_ords_prods.shape

(32404859, 23)

In [10]:
# dropping indicator column from previous merging of df_ords_prods dataset & aisle_id because we do not need it for our analysis
cols_to_drop = ['_merge', 'aisle_id']
df_ords_prods = df_ords_prods.drop(columns = cols_to_drop)

In [35]:
df_ords_prods.head(1)

,product_id,product_name,department_id,prices,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,...,reordered,price_range,busiest_day,busiest_hours,max_order,loyalty_flag,average_spent,spending_flag,order_frequency,order_frequency_flag
0,1,Chocolate Sandwich Cookies,19,5.8,3139998,138,28,6,11,3.0,...,0,Mid-range product,Regularly Busy,Most Orders,32,Regular Customer,6.935811,Low Spender,8.0,Frequent Customer


In [36]:
df_ords_prods.shape

(32404859, 21)

In [11]:
# merging the two datasets

df_combined = df_cust.merge(df_ords_prods, on = ['user_id'], indicator = True)

In [12]:
# exporting this dataset in parquet to save memory
df_combined.to_parquet(os.path.join(path, '2 Data', 'Prepared Data', 'cust_ords_prods_merge.parquet'), index = False)

In [13]:
df_combined.shape

(32404859, 31)

### Performing checks on our newly merged & exported dataset

In [3]:
df_final = pd.read_parquet(os.path.join(path, '2 Data', 'Prepared Data', 'cust_ords_prods_merge.parquet'))

In [4]:
df_final.columns

Index(['user_id', 'first_name', 'surname', 'gender', 'state', 'age',
       'date_joined', 'number_of_dependants', 'marital_status', 'income',
       'product_id', 'product_name', 'department_id', 'prices', 'order_id',
       'order_number', 'orders_day_of_the_week', 'order_hour_of_day',
       'days_since_prior_order', 'add_to_cart_order', 'reordered',
       'price_range', 'busiest_day', 'busiest_hours', 'max_order',
       'loyalty_flag', 'average_spent', 'spending_flag', 'order_frequency',
       'order_frequency_flag', '_merge'],
      dtype='object')

In [5]:
df_final['_merge'].value_counts(dropna = False)

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64